# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [19]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [50]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
496,YUM
497,ZBH
498,ZBRA
499,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [30]:
# from keySecret import IEX_CLOUD_API_TOKEN as token
# print (token)
token = 'sk_982c7c4917524e72b1d37e598c916c6e'

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [33]:
ticker = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{ticker}/quote/?token={token}'
data = requests.get(api_url).json()
print(data)


{'avgTotalVolume': 74661316, 'calculationPrice': 'close', 'change': 0.37, 'changePercent': 0.00246, 'close': 151.01, 'closeSource': 'official', 'closeTime': 1676062800366, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 151.03, 'delayedPriceTime': 1676062793796, 'extendedChange': 0.27, 'extendedChangePercent': 0.00179, 'extendedPrice': 151.28, 'extendedPriceTime': 1676077197973, 'high': 151.34, 'highSource': '15 minute delayed price', 'highTime': 1676062800089, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 151, 'iexCloseTime': 1676062798792, 'iexLastUpdated': 1676062798792, 'iexMarketPercent': 0.013236529652515336, 'iexOpen': 149.47, 'iexOpenTime': 1676039400218, 'iexRealtimePrice': 151, 'iexRealtimeSize': 283, 'iexVolume': 760448, 'lastTradeTime': 1676062799608, 'latestPrice': 151.01, 'latestSource': 'Close', 'latestTime': 'February 10, 2023', 'latestUpdate': 1676062800366, 'latestVolume': 57450708, 'low': 149.22, 'lowSource': '15 min

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [34]:
price = data['latestPrice']
market_cap = data['marketCap']

print(price , market_cap)

151.01 2389272065460


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [36]:
column_names = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
df = pd.DataFrame(columns = column_names)

In [37]:
df.append(
    pd.Series(
        [
            ticker,
            price,
            market_cap,
            'N/A'
        ],
    index = column_names
    ),
    ignore_index = True
)

/var/folders/zl/06q5s3dj1ls_3grfjl0n8pw80000gn/T/ipykernel_11911/2214568571.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,151.01,2389272065460,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [38]:
df = pd.DataFrame(columns = column_names)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://cloud.iexapis.com/stable/stock/{stock}/quote/?token={token}'
    data = requests.get(api_url).json()
    df = df.append(
        pd.Series(
            [
                stock,
                data['latestPrice'],
                data['marketCap'],
                'N/A'
            ],
            index = column_names
        ),
        ignore_index = True
    )

/var/folders/zl/06q5s3dj1ls_3grfjl0n8pw80000gn/T/ipykernel_11911/893478787.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/zl/06q5s3dj1ls_3grfjl0n8pw80000gn/T/ipykernel_11911/893478787.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/zl/06q5s3dj1ls_3grfjl0n8pw80000gn/T/ipykernel_11911/893478787.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/zl/06q5s3dj1ls_3grfjl0n8pw80000gn/T/ipykernel_11911/893478787.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
/var/folders/zl/06q5s3dj1ls_3grfjl0n8pw80000gn/T/ipykernel_11911/8934787

In [39]:
df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,152.55,45133454441,N/A
1,AAL,16.28,10580386440,N/A
2,AAP,151.80,9125948680,N/A
3,AAPL,151.01,2389272065460,N/A
4,ABBV,152.05,268897461241,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [40]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [52]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns = column_names)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={token}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = column_names), 
                                        ignore_index = True)
        
    

/var/folders/zl/06q5s3dj1ls_3grfjl0n8pw80000gn/T/ipykernel_11911/3426584146.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/zl/06q5s3dj1ls_3grfjl0n8pw80000gn/T/ipykernel_11911/3426584146.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/zl/06q5s3dj1ls_3grfjl0n8pw80000gn/T/ipykernel_11911/3426584146.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/zl/06q5s3dj1ls_3grfjl0n8pw80000gn/T/ipykernel_11911/3426584146.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fi

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [54]:
final_dataframe
final_dataframe.to_csv('recommended_trades.csv', index = False)

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: